In [1]:
import numpy as np
import re
import collections
import sklearn.preprocessing
import copy
vowels = list('aeioóuyąę') + list('aeioóuyąę'.upper())
compacted_vovels = ['i' + x for x in vowels if x != 'i']
def count_syllable(phrase, verose=False):
    res = 0
    for i, letter in enumerate(phrase):
        if letter in vowels:
            res += 1
            if verose:
                print(letter)
        if phrase[i:i+2] in compacted_vovels:
            res -= 1
            if verose:
                print(phrase[i:i+2])
    return res
def get_rym(w, verbose=False):
    best = None
    for i in range(len(w)):
        if count_syllable(w[i:], verbose) == 2:
            best = w[i:]
    return best

class NotFoundSimWords(Exception):
    pass

In [2]:
base_vectors = {}
with open('../data/poleval_base_vectors.txt') as f:
    f.readline()
    for i, x in enumerate(f):
        splitted = x.split()
        vec = np.array([[np.float64(x) for x in splitted[1:]]])
        sklearn.preprocessing.normalize(vec, copy=False)
        base_vectors[splitted[0]] = vec[0]
with open(f'../data/superbazy.txt') as f:
    superbase = dict([tuple(line.split()) for line in f])
with open(f'../data/supertags.txt') as f:
    supertags = dict([tuple(line.split()) for line in f])
# FILTER SUPETAGS
for word, tag in list(supertags.items()):
    if word not in superbase:
        del supertags[word]
        continue
    word_base = superbase[word]
    if word_base not in base_vectors:
        del supertags[word]
    
    
def same_supertags(tag):
    return (
        word for word, w_tag in supertags.items()
        if w_tag == tag
    )


        
with open('../data/rytmiczne_zdania_z_korpusu.txt') as f:
    sentences = [
        tuple(
            [
                [
                    x for x in y.split()
                ] 
                for y in line.split('RYM:')[1].rstrip(' .\n').split('[*]')
            ]
        ) 
        for line in f
        if 'POTENCJALNY_RYM' in line
    ]

In [3]:
def sample_verset():
    index = np.random.choice(np.arange(len(sentences)))
    return sentences[index]

def find_similar(goal_word, rym=None, accept_goal_word=False):
    goal_tag = supertags[goal_word]
    goal_syllabels = count_syllable(goal_word)
    return np.array([
        x
        for x in same_supertags(goal_tag)
        if rym is None or get_rym(x) == rym
        if count_syllable(x) == goal_syllabels
        if accept_goal_word or (x != goal_word)
    ])

def get_base_vector(word):
    base = superbase[word]
    return base_vectors[base]

def score_simmilarity(goal_word, words_set):
    words_set_vectors = np.array([get_base_vector(x) for x in words_set])
    return get_base_vector(goal_word) @ words_set_vectors.T    

def find_most_similar(words, scores, initial_word):
    best = words[scores.argmax()]
    return best

def format_output(output):
    for i, v in enumerate(output.values()):
        print(f'#####{i}')
        print(' '.join(v[0]))
        print(' '.join(v[1]))
        print()

In [4]:
def modify_middle_of_poem(verset):
    new_verset = copy.deepcopy(verset)
    for i, phrase in enumerate(verset):
        for j, word in enumerate(phrase[:-1]):
            if np.random.rand() < 0.6 and word in supertags:
                sim_set = find_similar(word)
#                 print(sim_set)
                if sim_set.size > 50:
                    scores = score_simmilarity(word, sim_set)
                    best_word = find_most_similar(sim_set, scores, word)
                    new_verset[i][j] = best_word
    return new_verset


In [5]:
def swich_ryhems(w1,w2):
    s_w1 = [(x, get_rym(x), get_base_vector(x)) for x in find_similar(w1, accept_goal_word=True)]
    s_w2 = [(x, get_rym(x), get_base_vector(x)) for x in find_similar(w2, accept_goal_word=True)]
    bv1 = get_base_vector(w1)
    bv2 = get_base_vector(w2)
    best = max((
        ((d1[2] @ bv1 + d2[2] @ bv2) / 2,
         d1[0],
         d2[0]
        )
        for d1 in s_w1
        for d2 in s_w2
        if d1[1] == d2[1]
    ))
    print(w1, w2)
    print(best[1], best[2])
    return best[1:]

def gen_poem():
    verset = sample_verset()
    new_verset = copy.deepcopy(verset)
    new_verset[0][-1], new_verset[1][-1] = swich_ryhems(verset[0][-1], verset[1][-1])
    res = {
        'initial': verset,
        'changed': new_verset,
    }
    # format_output(res)
    res['3'] = modify_middle_of_poem(res['changed'])
    format_output(res)

In [6]:
gen_poem()

otrzymuje wypracowań
marnotrawi prawosławi
#####0
dodatkowe oceny uczeń otrzymuje
za niektóre z pisanych w domu wypracowań

#####1
dodatkowe oceny uczeń marnotrawi
za niektóre z pisanych w domu prawosławi

#####2
przestojowe oceny uczeń marnotrawi
za niektóre z pisanych w domu prawosławi



In [7]:
gen_poem()

gatunki ozdobne
pływacze kozacze
#####0
gdzie indziej , także w polsce niektóre gatunki
są uprawiane jako rośliny ozdobne

#####1
gdzie indziej , także w polsce niektóre pływacze
są uprawiane jako rośliny kozacze

#####2
gdzie indziej , także w polsce niektóre pływacze
są uprawiane jako krzewinki kozacze



In [8]:
gen_poem()

minęła dzieła
zleciała ciała
#####0
wacław trzeba więc jechać , bo moda minęła
, z piątego aktu sądzić o dobroci dzieła

#####1
wacław trzeba więc jechać , bo moda zleciała
, z piątego aktu sądzić o dobroci ciała

#####2
zygmunt trzeba więc jeździć , bo moda zleciała
, z piątego aktu wierzyć o bojaźni ciała

